##### 问题:
我们希望通过函数来扩展闭包，使得在闭包内层定义的变量可以被访问和修改。

##### 解决方案:
一般来说，在闭包内层定义的变量对于外界来说完全是隔离的。但是，可以通过编写
存取函数（accessor function，即 getter/setter 方法）并将它们作为函数属性附加到闭包
上来提供对内层变量的访问支持。示例如下：

In [45]:
def sample():
    n = 0
 # Closure function
    def func():
        print('n=', n)
    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func 



In [46]:
f = sample()
f() 
f.set_n(10)     
f() 
f.get_n() 


n= 0
n= 10


10

这里主要用到了两个特性使得本节讨论的技术得以成功实施。首先，nonlocal 声明使得
编写函数来修改内层变量成为可能。其次，函数属性能够将存取函数以直接的方式附
加到闭包函数上，它们工作起来很像实例的方法（尽管这里并没有涉及类）。

对本节提到的技术稍作扩展就可以让闭包模拟成类实例。我们所要做的就是将内层函
数拷贝到一个实例的字典中然后将它返回。示例如下：

In [47]:
import sys
class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals
        # Update instance dictionary with callables
        self.__dict__.update((key,value) for key, value in locals.items() if callable(value) )

 # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()
    # Example use

def Stack():
    items = []
    def push(item):
        items.append(item)
    def pop():
        return items.pop()
    def __len__():
        return len(items)
    return ClosureInstance()

In [48]:
s = Stack() 
print(s)
s.push(10)
s.push(20)
s.push('Hello')
print(len(s)) 
print(s.pop())
print(s.pop())
print(s.pop())

3
Hello
20
10


有趣的是，这份代码运行起来比使用一个普通的类定义要稍微快一些。比如，我们可
能会用下面这个类来做对比测试：

In [49]:
class Stack2:
    def __init__(self):
        self.items = []
    def push(self, item):
        self.items.append(item)
    def pop(self):
        return self.items.pop()
    def __len__(self):
        return len(self.items)

如果进行对比测试，将得到类似如下的结果：

In [50]:
from timeit import timeit
# Test involving closures
s = Stack()
print(timeit('s.push(1);s.pop()', 'from __main__ import s'))

# Test involving a class
s = Stack2()
print(timeit('s.push(1);s.pop()', 'from __main__ import s'))

1.8519595000000209
1.912212899999986


我们可以看到，采用闭包的版本要快大约 8%。测试中的大部分时间都花在对实例变量
的直接访问上，闭包要更快一些，这是因为不用涉及额外的 self 变量。

Raymond Herringer 在这个思路的基础上设计出了一种更加“恐怖”的变种。但是，在
自己的代码中应该对这种奇技淫巧持谨慎的态度。请注意，相比一个真正的类，这种
方法是相当怪异的。比如，像继承、属性、描述符或者类方法这样的主要特性在这种
方法中都是无法使用的。我们还需要玩一些花招才能让特殊方法正常工作（比如，参
考 ClosureInstance 中对__len__()的实现）。

最后，这么做会使得阅读你代码的人犯糊涂。他们会想知道这么做看起来和一个普通
的类定义相比有什么区别（当然了，他们也想知道为什么这么做会运行的更快一些）。
尽管如此，这仍然是个有趣的例子，它告诉我们对闭包内部提供访问机制能够实现出
什么样的功能。

从全局的角度考虑，为闭包增加方法可能会有着更多的实际用途，比如我们想重置内
部状态、刷新缓冲区、清除缓存或者实现某种形式的反馈机制（feedback mechanism）。